In [ ]:
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None

In [ ]:
FOLDER = './'
df_raw = pd.read_csv(FOLDER + 'blocks.csv')
df_raw.info()
df_raw[df_raw.duplicated(['block_number'], keep=False)]

In [ ]:
df_raw.drop_duplicates(['block_number'], keep='last', inplace=True)
df_raw.sort_values(by='block_number', ascending=True, inplace=True)
df_raw.reset_index(drop=True, inplace=True)

df_raw.info()

In [ ]:
#takes a list and allocates each value to a bin
lst = np.array(df_raw['block_number'])

start = np.round(df_raw['block_number'].min(), -2)
end = np.round(df_raw['block_number'].max(), -2)

#creates bins for each 32nd block (slot)
bins = np.arange(np.round(df_raw['block_number'].min(), -2), np.round(df_raw['block_number'].max(), -2), 32)

temp = []
for i in range(1,len(bins)+1):
    temp.append(lst[np.digitize(lst,bins)==i])


In [ ]:
#connects slot and blocks within slots
data = list(zip(bins, temp))
data

In [ ]:
fig, ax = plt.subplots(figsize =(20,4))

x = [ele[0] for ele in data]
y = [len(ele[1]) for ele in data]

ax.scatter(x, y, s=20, alpha=.6)

ax.set_ylabel('amount of flashbots blocks')

plt.yticks(range(0,20+1,4))

txt = 'The figure displays the number of flashbots blocks within each slot.'
fig.text(.5, -.1, txt, ha='center', fontstyle='italic')

ax.set(xlim = (start-400, end+400))

xlabels = ['{:,.3f}'.format(x/1e6) + 'mn' for x in ax.get_xticks()]
ax.set_xticklabels(xlabels)


#plt.savefig('t.png')
plt.show()

In [ ]:
#calculates consecutive blocks

temp = df_raw['block_number'].values

from itertools import groupby
from operator import itemgetter

#enummerate creates tuple(element index, element value); (0,24),(1,25)
#lambda substracts: element index - element value; (-24)(-24)
#groupby groups the results with equal value
#map(fct, lst) applies a function to every item of the list; equal to apply for dataframes

con_blocks = []

for k,g in groupby(enumerate(temp), lambda x:x[0]-x[1]):
    group = map(itemgetter(1),g)
    con_blocks.append(list(map(int,group)))

con_blocks

In [ ]:
#takes the lowest blocknumber of each consecutive block and calculates the length
temp = []
for i in range(0, len(con_blocks)):
    temp.append((con_blocks[i][0], len(con_blocks[i])))

temp

In [ ]:
#takes the list (first blocknumber of consec. blocks) and allocates it to the bin
lst = np.array([ele[0] for ele in temp])
bins = bins
#however it attaches the lengths of the consec. blocks
cons_block = np.array([ele[1] for ele in temp])

temp = []
for i in range(1,len(bins)+1):
    temp.append(cons_block[np.digitize(lst,bins)==i])

temp

In [ ]:
#unzip the list
bins, cons_blocks = zip(*data)

In [ ]:
data = list(zip(bins, cons_blocks, temp))
data

In [ ]:
fig, ax = plt.subplots(figsize =(20,8))

#scatters multiple times; takes the first and third argument in data
for u,_,t in data:
    #scale x to number of consec. blocks
    x = [u] * len(t)
    y = t
    y_jittered = y + np.random.uniform(-.3,.3)
    plt.scatter(x, y_jittered, s=20, alpha=.4, c='b')

ax.set_ylabel('amount of flashbots blocks')

plt.yticks(range(0,8+1,1))

ax.set(xlim = (start-400, end+400))

txt = 'The figure displays the number of flashbots blocks within each slot.'
fig.text(.5, -.1, txt, ha='center', fontstyle='italic')

xlabels = ['{:,.3f}'.format(x/1e6) + 'mn' for x in ax.get_xticks()]
ax.set_xticklabels(xlabels)

#plt.savefig('t.png')
plt.show()


